In [1]:
from transformers import pipeline

/home/luis-carlos/Documents/experiments/llm_env/lib/python3.12/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-06-22 21:36:21.632698: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-22 21:36:22.900014: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/luis-carlos/Documents/experiments/llm_env/lib/python3.12/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
pipe = pipeline(model="facebook/bart-large-mnli")

/home/luis-carlos/Documents/experiments/llm_env/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
pipe("I have a problem with my iphone that needs to be resolved asap!",
    candidate_labels=["urgent", "not urgent", "phone", "tablet", "computer"],
)

{'sequence': 'I have a problem with my iphone that needs to be resolved asap!',
 'labels': ['urgent', 'phone', 'computer', 'not urgent', 'tablet'],
 'scores': [0.5227578282356262,
  0.4581405818462372,
  0.014264578931033611,
  0.0026850118301808834,
  0.0021520578302443027]}

## Part 2.

In [4]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [5]:
text = "I enjoy playing cricket, specializing as a left-arm leg spinner while showcasing my skills as a right-handed one-down batsman." 
labels = ['Politics', 'Automobile', 'Sports', 'Business', 'World']

prediction = classifier(text, labels)  

print(prediction['sequence']) 
print(prediction['labels']) 
print(prediction['scores'])

I enjoy playing cricket, specializing as a left-arm leg spinner while showcasing my skills as a right-handed one-down batsman.
['Sports', 'World', 'Automobile', 'Business', 'Politics']
[0.9363625645637512, 0.051277656108140945, 0.005267278756946325, 0.0048265582881867886, 0.002265882445499301]


# Downloading dataset

In [6]:
import pandas as pd
from glob2 import glob

In [7]:
if 'customer_support.csv' not in glob('*'):
    df = pd.read_csv("hf://datasets/bitext/Bitext-customer-support-llm-chatbot-training-dataset/Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv")
    df.to_csv('customer_support.csv', index=False)
    df.head()
else:
    df = pd.read_csv('customer_support.csv')

df.head()

,flags,instruction,category,intent,response
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


In [8]:
df['category'].unique()

array(['ORDER', 'SHIPPING', 'CANCEL', 'INVOICE', 'PAYMENT', 'REFUND',
       'FEEDBACK', 'CONTACT', 'ACCOUNT', 'DELIVERY', 'SUBSCRIPTION'],
      dtype=object)

## NER

In [9]:
import spacy
from spacy import displacy

In [10]:
# python -m spacy download en_core_web_sm

In [11]:
NER = spacy.load("en_core_web_sm")

In [12]:
raw_text="The Indian Space Research Organisation or is the national space agency of India, headquartered in Bengaluru. It operates under Department of Space which is directly overseen by the Prime Minister of India while Chairman of ISRO acts as executive of DOS as well."

In [13]:
text1 = NER(raw_text)

In [14]:
for word in text1.ents:
    print(word.text,word.label_)

The Indian Space Research Organisation ORG
India GPE
Bengaluru GPE
Department of Space ORG
India GPE
ISRO ORG
DOS ORG


In [15]:
spacy.explain("ORG")

'Companies, agencies, institutions, etc.'

In [16]:
displacy.render(text1,style="ent",jupyter=True)

### Testing personal information

Wanderlust Wayfarer: Hello! Welcome to Wanderlust Wayfarer, your one-stop shop for crafting the perfect getaway.  Where in the world have you always dreamed of traveling?

User: Hi there! I've been wanting to visit Europe for a while, but there are so many places to choose from!

Wanderlust Wayfarer:  Excellent taste! Europe offers a vast array of experiences. To narrow it down a bit, what kind of atmosphere are you drawn to?  Are you looking for a relaxing beach vacation, a historical and cultural adventure, or perhaps a bustling city experience?

User: Leaning more towards relaxation, with some beautiful scenery. Maybe somewhere with gorgeous beaches and historical sites I can visit at my own pace.

Wanderlust Wayfarer:  Marvelous!  The Greek Islands might be perfect for you.  They offer stunning beaches, charming villages steeped in history, and a relaxed pace of life.  Would you be traveling solo, with a partner, or with a larger group?

User: It would be a trip with my partner. We've been wanting a romantic getaway to celebrate our anniversary.

Wanderlust Wayfarer:  A lovely plan!  Knowing it's a couple's trip allows me to recommend some charming boutique hotels or secluded beachfront villas.  When were you hoping to travel? Knowing the timeframe helps with flight and accommodation options.

User (after checking a calendar): Ideally, we'd love to go sometime in September.

Wanderlust Wayfarer: Perfect! September is a shoulder season in many European destinations, so you might find better deals and slightly fewer crowds.  To personalize your experience further, would you be comfortable sharing a rough budget range for the trip?

User: Absolutely! We were thinking somewhere in the ballpark of $5,000 per person.

Wanderlust Wayfarer:  Fantastic!  With all this information, I can curate a selection of Greek island getaways that perfectly match your desires.  Would you like me to find some romantic restaurants or unique experiences to add to your itinerary as well?

This example shows the chatbot collecting various details that could be considered personal information:

    Travel preferences (dream destinations, desired atmosphere)
    Travel companions (solo, couple, group)
    Reason for travel (celebration)
    Preferred travel dates
    Budget range

While none of this information is super sensitive on its own, it paints a picture of the user's identity and travel habits.  This allows the chatbot to provide a more personalized service.  It's important to note that reputable chatbots will usually clearly state their privacy policy and how they handle user data.

In [17]:
raw_text = "Hi there! I've been wanting to visit Europe for a while, but there are so many places to choose from!"

In [18]:
def analyze_text(text):
    doc = NER(text)
    for word in doc.ents:
        print(word.text, word.label_)
    return doc

In [19]:
res = analyze_text(text=raw_text)

Europe LOC


In [20]:
res = analyze_text(text = "Leaning more towards relaxation, with some beautiful scenery. Maybe somewhere with gorgeous beaches and historical sites I can visit at my own pace.")

In [76]:
raw_text = "It would be a trip with my partner. We've been wanting a romantic getaway to celebrate our anniversary."
res = analyze_text(text = raw_text)
res

It would be a trip with my partner. We've been wanting a romantic getaway to celebrate our anniversary.

In [77]:
def predict_class(text, labels, print_summary=False):
    prediction = classifier(text, labels)

    tmp = pd.DataFrame([prediction['labels'], prediction['scores']]).T
    tmp.columns = ['labels', 'scores']

    if print_summary:
        print(text)
        display(tmp)
    return prediction

In [78]:
labels = ['solo', 'couple', 'group']
res = predict_class(raw_text, labels, print_summary=True)

It would be a trip with my partner. We've been wanting a romantic getaway to celebrate our anniversary.


,labels,scores
0,couple,0.877652
1,group,0.110046
2,solo,0.012303


In [79]:
labels = ['Leisure', 'Business', 'Education', 'Visiting Friends and Family', 'Medical', 'Adventure', 'Workation', 'celebration']
res = predict_class(raw_text, labels, print_summary=True)

It would be a trip with my partner. We've been wanting a romantic getaway to celebrate our anniversary.


,labels,scores
0,celebration,0.861152
1,Leisure,0.048936
2,Adventure,0.034239
3,Workation,0.020371
4,Education,0.010071
5,Visiting Friends and Family,0.009811
6,Medical,0.009387
7,Business,0.006032


## Testing with a dataset

In [90]:
from sklearn.utils import shuffle
from tqdm import tqdm
import re

In [81]:
df.head()

,flags,instruction,category,intent,response
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


In [124]:
sample_size = 10
tmpDF = df.groupby('category').sample(n=sample_size, replace=True)
tmpDF.head()

,flags,instruction,category,intent,response
11680,BKLZ,remove {{Account Category}} accoint,ACCOUNT,delete_account,Thank you for contacting us regarding the remo...
21881,BILM,what do I have to do to report issues with a s...,ACCOUNT,registration_problems,Appreciate your message to us about the issues...
11119,BLMQ,i have problems with the deletion of a {{Accou...,ACCOUNT,delete_account,Glad you contacted to us about the issues you ...
20830,BCILZ,"I forgot my account password, how t reset it?",ACCOUNT,recover_password,"Oh, I completely understand how frustrating it..."
20587,BLQ,i want help recovering the access key of my pr...,ACCOUNT,recover_password,I realized you are seeking assistance in recov...


In [125]:
tmpDF.shape

(110, 5)

In [126]:
tmpDF = shuffle(tmpDF)
tmpDF.reset_index(drop=True, inplace=True)
tmpDF.head()

,flags,instruction,category,intent,response
0,BKL,cancelling {{Account Category}} account,ACCOUNT,delete_account,I appreciate you reaching out to us regarding ...
1,BIL,where do I make a customer reclamation against...,FEEDBACK,complaint,I sincerely apologize for any inconvenience yo...
2,BLQ,i want assistance to swap something of purchas...,ORDER,change_order,We appreciate your request to swap an item in ...
3,BLQ,i need help to see the early termination charge,CANCEL,check_cancellation_fee,"Sure! To view the early termination charge, yo..."
4,BILMW,how do I look for the fucking invoices from {{...,INVOICE,check_invoice,I apologize if there was any confusion in your...


In [127]:
labels = tmpDF['category'].unique().tolist()
labels

['ACCOUNT',
 'FEEDBACK',
 'ORDER',
 'CANCEL',
 'INVOICE',
 'DELIVERY',
 'SUBSCRIPTION',
 'CONTACT',
 'SHIPPING',
 'PAYMENT',
 'REFUND']

In [128]:
precision = []

for row in tqdm(tmpDF.itertuples(), total=len(tmpDF)):
    raw_text = row.instruction
    prediction = predict_class(raw_text, labels, print_summary=False)
    precision.append(prediction['labels'][0] == row.category)

100%|██████████| 110/110 [09:05<00:00,  4.96s/it]


In [129]:
precision[0:20]

[False,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True]

In [130]:
total_precision = sum(precision) / len(precision)
print(f'Total accuracy: {total_precision * 100:.2f}%')

Total accuracy: 52.73%


In [131]:
n=0
print(tmpDF.loc[n, 'category'])
predict_class(tmpDF.loc[n, 'instruction'], labels, print_summary=True)

ACCOUNT
cancelling {{Account Category}} account


,labels,scores
0,CANCEL,0.737349
1,ACCOUNT,0.132713
2,INVOICE,0.035339
3,ORDER,0.020385
4,CONTACT,0.018864
5,PAYMENT,0.011521
6,REFUND,0.011154
7,FEEDBACK,0.010736
8,SUBSCRIPTION,0.007538
9,SHIPPING,0.007288


{'sequence': 'cancelling {{Account Category}} account',
 'labels': ['CANCEL',
  'ACCOUNT',
  'INVOICE',
  'ORDER',
  'CONTACT',
  'PAYMENT',
  'REFUND',
  'FEEDBACK',
  'SUBSCRIPTION',
  'SHIPPING',
  'DELIVERY'],
 'scores': [0.7373490929603577,
  0.13271336257457733,
  0.035339076071977615,
  0.020384525880217552,
  0.018863528966903687,
  0.0115209324285388,
  0.011154357343912125,
  0.010736397467553616,
  0.0075379349291324615,
  0.007288334425538778,
  0.007112595252692699]}

In [132]:
n=1
print(tmpDF.loc[n, 'category'])
predict_class(tmpDF.loc[n, 'instruction'], labels, print_summary=True)

FEEDBACK
where do I make a customer reclamation against your company?


,labels,scores
0,REFUND,0.520703
1,CONTACT,0.098496
2,INVOICE,0.08496
3,FEEDBACK,0.082622
4,ORDER,0.06257
5,ACCOUNT,0.045967
6,PAYMENT,0.031589
7,DELIVERY,0.028536
8,SUBSCRIPTION,0.015488
9,SHIPPING,0.014954


{'sequence': 'where do I make a customer reclamation against your company?',
 'labels': ['REFUND',
  'CONTACT',
  'INVOICE',
  'FEEDBACK',
  'ORDER',
  'ACCOUNT',
  'PAYMENT',
  'DELIVERY',
  'SUBSCRIPTION',
  'SHIPPING',
  'CANCEL'],
 'scores': [0.5207028388977051,
  0.09849632531404495,
  0.0849602147936821,
  0.08262209594249725,
  0.06256954371929169,
  0.04596659541130066,
  0.03158948943018913,
  0.028535865247249603,
  0.015488037839531898,
  0.014953576028347015,
  0.014115436002612114]}

# Snorkel
## Preprocessing

In [176]:
import nltk.corpus
#nltk.download('stopwords')
from nltk.corpus import stopwords
import numpy as np

In [134]:
text = tmpDF.loc[0, 'instruction']
print(text)

cancelling {{Account Category}} account


In [144]:
def clean_text(text):
    # Normalizing
    text = text.lower()

    # Remove everything inside parentheses and the parentheses themselves
    text = re.sub(r'\s* \{\{.*?\}\}\s* ', ' ', text)

    # Removing numbers
    text = re.sub(r"[0-9]+", "", text)

    # Removing characters
    text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", " ", text)

    # Removing stopwords
    stop = stopwords.words('english')
    text = " ".join([word for word in text.split() if word not in stop])
    return(text)

In [145]:
tmpDF.head()

,flags,instruction,category,intent,response,instruction_cleaned
0,BKL,cancelling {{Account Category}} account,ACCOUNT,delete_account,I appreciate you reaching out to us regarding ...,cancelling account
1,BIL,where do I make a customer reclamation against...,FEEDBACK,complaint,I sincerely apologize for any inconvenience yo...,make customer reclamation company
2,BLQ,i want assistance to swap something of purchas...,ORDER,change_order,We appreciate your request to swap an item in ...,want assistance swap something purchase order ...
3,BLQ,i need help to see the early termination charge,CANCEL,check_cancellation_fee,"Sure! To view the early termination charge, yo...",need help see early termination charge
4,BILMW,how do I look for the fucking invoices from {{...,INVOICE,check_invoice,I apologize if there was any confusion in your...,look fucking invoices person name


In [146]:
tmpDF['instruction_cleaned'] = tmpDF['instruction'].apply(clean_text)
tmpDF.tail()

,flags,instruction,category,intent,response,instruction_cleaned
105,BILM,what do I have to do to report issues with a s...,ACCOUNT,registration_problems,Appreciate your message to us about the issues...,report issues sign
106,BKLMZ,reimbusre {{Currency Symbol}}{{Refund Amount}},REFUND,get_refund,I've decoded that you would like to inquire ab...,reimbusre currency symbol refund amount
107,BLMQ,need to inform of issues with payment,PAYMENT,payment_issue,I appreciate you reaching out to inform us abo...,need inform issues payment
108,BL,I don't know how I can see when will my item a...,DELIVERY,delivery_period,No worries at all! I'm here to help you with t...,know see item arrive
109,BL,I am trying to speak to somebody,CONTACT,contact_human_agent,Always good to connect! I'm attuned to the fac...,trying speak somebody


In [148]:
for label in labels:
    labelDF = tmpDF[tmpDF['category']==label]
    allList = labelDF['instruction_cleaned'].values.tolist()

    a = [text.split() for text in allList]
    joined_list = [word for sublist in a for word in sublist]

    print(label)
    print(pd.Series(joined_list).value_counts().head(10))
    print('--------------\n')

ACCOUNT
account       5
reset         2
profile       2
report        2
sign          2
problems      2
cancelling    1
want          1
gold          1
change        1
Name: count, dtype: int64
--------------

FEEDBACK
help           4
feedback       4
make           3
reclamation    3
company        3
need           3
file           2
complaint      2
ur             2
customer       2
Name: count, dtype: int64
--------------

ORDER
order         7
number        6
purchase      6
want          3
status        3
help          3
cancel        2
assistance    2
seeing        2
swap          2
Name: count, dtype: int64
--------------

CANCEL
termination     5
check           5
charges         4
see             4
early           4
charge          4
help            4
need            2
fee             2
cancellation    2
Name: count, dtype: int64
--------------

INVOICE
person        6
name          6
bill          5
want          3
assistance    3
look          2
need          2
find        

In [149]:
from snorkel.labeling import labeling_function

In [150]:
ABSTAIN = -1
ACCOUNT = 0
FEEDBACK = 1
ORDER = 2

In [171]:
tmpDF2 = tmpDF[tmpDF.category.isin(['ACCOUNT', 'FEEDBACK', 'ORDER'])].reset_index(drop=True)
tmpDF2.head()

,flags,instruction,category,intent,response,instruction_cleaned
0,BKL,cancelling {{Account Category}} account,ACCOUNT,delete_account,I appreciate you reaching out to us regarding ...,cancelling account
1,BIL,where do I make a customer reclamation against...,FEEDBACK,complaint,I sincerely apologize for any inconvenience yo...,make customer reclamation company
2,BLQ,i want assistance to swap something of purchas...,ORDER,change_order,We appreciate your request to swap an item in ...,want assistance swap something purchase order ...
3,BLQ,help to file a complaint against ur company,FEEDBACK,complaint,I deeply regret any negative experience you've...,help file complaint ur company
4,BILQ,how can I reset my profile pass?,ACCOUNT,recover_password,I'm happy to help! I'm here to shed light on r...,reset profile pass


In [172]:
# Define the labeling function for account
@labeling_function()
def check_account(x):
    text = x.text.lower()
    strings = ["account", "reset", "profile", "report", "sign", "problems"]
    for string in strings:
        if string in text:
            return ACCOUNT
    return ABSTAIN


# Define the labeling function for feedback
@labeling_function()
def check_feedback(x):
    text = x.text.lower()
    strings = ["help", "feedback", "make", "reclamation", "company", "need", "complaint", "customer"]
    for string in strings:
        if string in text:
            return FEEDBACK
    return ABSTAIN


# Define the labeling function for feedback
@labeling_function()
def check_order(x):
    text = x.text.lower()
    strings = ["order", "number", "purchase", "want", "status", "help", "cancel", "assistance", "seeing", "swap"]
    for string in strings:
        if string in text:
            return ORDER
    return ABSTAIN

In [179]:
df_train = tmpDF2[['instruction_cleaned', 'category']].copy()
df_train.rename(columns={'instruction_cleaned': 'text', 'category': 'label'}, inplace=True)
df_train.head()

,text,label
0,cancelling account,ACCOUNT
1,make customer reclamation company,FEEDBACK
2,want assistance swap something purchase order ...,ORDER
3,help file complaint ur company,FEEDBACK
4,reset profile pass,ACCOUNT


In [180]:
from snorkel.labeling import PandasLFApplier

lfs = [check_account, check_feedback, check_order]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

100%|██████████| 30/30 [00:00<00:00, 3449.74it/s]


In [181]:
L_train

array([[ 0, -1,  2],
       [-1,  1, -1],
       [-1, -1,  2],
       [-1,  1,  2],
       [ 0, -1, -1],
       [-1,  1,  2],
       [-1, -1,  2],
       [-1,  1,  2],
       [-1, -1,  2],
       [-1,  1,  2],
       [-1,  1, -1],
       [-1,  1, -1],
       [ 0, -1, -1],
       [-1, -1, -1],
       [-1,  1,  2],
       [-1, -1,  2],
       [ 0, -1, -1],
       [-1, -1,  2],
       [-1,  1,  2],
       [-1,  1, -1],
       [-1,  1, -1],
       [ 0, -1, -1],
       [-1,  1,  2],
       [-1, -1,  2],
       [-1,  1,  2],
       [ 0, -1, -1],
       [-1, -1,  2],
       [ 0,  1,  2],
       [ 0,  1, -1],
       [ 0, -1, -1]])

In [182]:
np.argmax(L_train, axis=1)

array([2, 1, 2, 2, 0, 2, 2, 2, 2, 2, 1, 1, 0, 0, 2, 2, 0, 2, 2, 1, 1, 0,
       2, 2, 2, 0, 2, 2, 1, 0])

In [190]:
df_train

,text,label
0,cancelling account,ACCOUNT
1,make customer reclamation company,FEEDBACK
2,want assistance swap something purchase order ...,ORDER
3,help file complaint ur company,FEEDBACK
4,reset profile pass,ACCOUNT
5,help lodge customer reclamation,FEEDBACK
6,cannot poay purchase order number,ORDER
7,need assistance sending feedback service,FEEDBACK
8,want assistance acquiring ur article,ORDER
9,need help send feedback product,FEEDBACK
